# Alternate Z02FH Analysis Approach

In [44]:
import re
def ExtractTestCase(x):
    return re.search("_D(\d{3})",x)[1]
    
print(ExtractTestCase('MB_U01_MZ02F_D420'))


def Maccor_parser(x):
    days = float(re.search("(.+?)d",x)[1])
    hours= re.search("(?<=d)(.*)",x)[1].strip()
    h =    float(re.search("(\d*)(?=:)",hours)[1])
    m =    float(re.search("(?<=:)(\d*)",hours)[1])
    s =    float(re.search("(?<=:\d\d:)(.*)",hours)[1])
    return days*1440+h*60+m+s/60

print(Maccor_parser('4d 22:25:6.53000020980835')) # Test pattern

420
7105.10883333683


In [71]:
# This is an intersting alternate approach for reading in the battery data
import glob
import os
import pandas as pd
from pathlib import Path
from functools import partial

prs = partial(pd.read_csv, 
              sep="\t", 
              header=0, 
              skiprows=4, 
              usecols=['Rec#','Cyc#','Step','TestTime','StepTime','Amp-hr','Amps','Volts','State']
             )
path=os.path.join(os.getcwd(), "Z02_Test")  # This reminds me of what I have to do with PQ
files = Path(path).glob('*.txt')            # Making a path object provides all sorts of great methods

df = pd.concat((prs(f).assign(filename=f.stem) for f in files), ignore_index=True)
df.StepTime=df.StepTime.apply(Maccor_parser)
df.TestTime=df.TestTime.apply(Maccor_parser)
df.filename=df.filename.apply(ExtractTestCase)
df.rename(columns={"filename": "Test_Case","Rec#":"Rec","Cyc#":"Cyc"},inplace=True)
df=df.set_index('Rec')
#df.info()

In [70]:
df

,Cyc,Step,TestTime,StepTime,Amp-hr,Amps,Volts,State,Test_Case
Rec,,,,,,,,,
1,0,1,0.000000,0.000000,0.00000,0.0000,3.79103,R,420
2,0,1,1.000000,1.000000,0.00000,0.0000,3.79118,R,420
3,1,4,1.000000,0.000000,0.00000,0.0005,3.78630,P,420
4,1,5,1.000167,0.000167,0.00000,0.0000,3.79042,D,420
5,1,5,2.000167,1.000167,0.00001,0.0005,3.78233,D,420
...,...,...,...,...,...,...,...,...,...
7139,5,5,7105.108833,738.000667,0.00615,0.0005,3.85199,D,435
7140,5,5,7106.108833,739.000667,0.00616,0.0005,3.85168,D,435
7141,5,5,7107.108833,740.000667,0.00617,0.0005,3.85130,D,435
